In [3]:
# local machine only
import sys
sys.path.append('/home/bp/devel/mupif.git')
sys.path.append('.')

import mupif as mp
import indirectAPI
import models

In [4]:
import mupif
import numpy as np
import subprocess
class myAPI(mupif.model.Model):
    """
    Simple application that computes an arithmetical average of mapped property using an external code
    """
    def __init__(self, metadata={}):
        if len(metadata) == 0:
            metadata = {
                'Name': 'My application API',
                'ID': 'MyApp 1.0',
                'Description': 'Computes some characteristic of a field',
                'Version_date': '1.0.0, Feb 2019',
                'Geometry': '2D rectangle',
                'Inputs': [
                    {
                        'Name': 'scalar field',
                        'Type': 'mupif.Field',
                        'Required': True,
                        'Type_ID': 'mupif.PropertyID.PID_Temperature',
                    }
                ],
                'Outputs': [
                    {
                        'Name': 'min',
                        'Type_ID': 'mupif.PropertyID.PID_Demo_Min',
                        'Type': 'mupif.Property',
                        'Required': False
                    },
                    {
                        'Name': 'max',
                        'Type_ID': 'mupif.PropertyID.PID_FieldID.FID_Temperature',
                        'Type': 'mupif.Property',
                        'Required': False
                    },
                    {
                        'Name': 'average',
                        'Type_ID': 'mupif.PropertyID.PID_Demo_Value',
                        'Type': 'mupif.Property',
                        'Required': False
                    }
                ],
                'Solver': {
                    'Software': 'own',
                    'Type': 'none',
                    'Accuracy': 'Medium',
                    'Sensitivity': 'Low',
                    'Complexity': 'Low',
                    'Robustness': 'High',
                    'Estim_time_step_s': 1,
                    'Estim_comp_time_s': 1.e-3,
                    'Estim_execution_cost_EUR': 0.01,
                    'Estim_personnel_cost_EUR': 0.01,
                    'Required_expertise': 'None',
                    'Language': 'Python',
                    'License': 'LGPL',
                    'Creator': 'Borek Patzak',
                    'Version_date': '1.0.0, Feb 2019',
                    'Documentation': 'none',
                },
                'Physics': {
                    'Type': 'Continuum',
                    'Entity': ['Finite volume'],
                    'Equation': ['none'],
                    'Equation_quantities': ['Temperature'],
                    'Relation_description': [],
                    'Relation_formulation': [],
                    'Representation': ''
                }
            }
        super().__init__(metadata=metadata)
        self.min = None
        self.max = None
        self.average = None
        self.field = None
    def getProperty(self, propID, time, objectID=0):
        if (propID == mupif.PropertyID.PID_Demo_Min):
           return mupif.property.ConstantProperty(value=self.min, propID=mupif.PropertyID.PID_Demo_Min, valueType=mupif.ValueType.Scalar, unit=mupif.U['K'])
        elif (propID == mupif.PropertyID.PID_Demo_Max):
           return mupif.property.ConstantProperty(value=self.max, propID=mupif.PropertyID.PID_Demo_Max, valueType=mupif.ValueType.Scalar, unit=mupif.U['K'])
        elif (propID == mupif.PropertyID.PID_Demo_Value):
           return mupif.property.ConstantProperty(value=self.average, propID=mupif.PropertyID.PID_Demo_Value, valueType=mupif.ValueType.Scalar, unit=mupif.U['K'])
        else:
           raise mupif.APIError.APIError ('Unknown property ID')
    def setField(self, field, objectID=0):
        if (field.getFieldID() == mupif.FieldID.FID_Temperature):
            # remember the mapped value
            self.field = field
        else:
            raise mupif.APIError.APIError ('Unknown field ID')
    def solveStep(self, tstep, stageID=0, runInBackground=False):
        if (self.field):
            input = ""
            for x in np.linspace(0,5,20):
                for y in np.linspace(0,1,5):
                    input+=repr(self.field.evaluate((x,y,0)).getValue()[0])
                    input +="\n"
        # We create the first subprocess, note that we need stdin=PIPE and stdout=PIPE
        p1 = subprocess.Popen(['python3', 'externalApp.py'], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

        # We immediately run the first subprocess and get the result
        # Note that we encode the data, otherwise we'd get a TypeError
        p1_out = p1.communicate(input=input.encode())
        
        tokens=p1_out[0].split()
        self.min = float(tokens[0])
        self.max = float(tokens[1])
        self.average = float(tokens[2])

In [5]:
# local machine only
import sys
sys.path.append('/home/bp/devel/mupif.git')
sys.path.append('.')

import mupif as mp
import indirectAPI
import models

ts = models.ThermalModel()
m2 = indirectAPI.myAPI()

ts.initialize('inputT.in')
bc1 = mp.ConstantProperty(propID=mp.PropertyID.PID_Temperature, valueType=mp.ValueType.Scalar, value=(50,), unit=mp.U['K'])
bc2 = mp.ConstantProperty(propID=mp.PropertyID.PID_Temperature, valueType=mp.ValueType.Scalar, value=(0,), unit=mp.U['K'])
ts.set(bc1, 'Cauchy left')
ts.set(bc2, 'Cauchy right')
ts.solveStep(mp.TimeStep(time=0,dt=.1,targetTime=1.,unit=mp.U.s))
# show the field
f=ts.getField(fieldID=mp.FieldID.FID_Temperature,time=1.*mp.Q.s)

m2.set (f)
m2.solveStep(mp.TimeStep(time=0,dt=.1,targetTime=1.,unit=mp.U.s))
print(m2.get(mp.PropertyID.PID_Demo_Min))
print(m2.get(mp.PropertyID.PID_Demo_Max))
print(m2.get(mp.PropertyID.PID_Demo_Value))



7.142857142857114{K,PropertyID.PID_Demo_Min,ValueType.Scalar}@None
42.85714285714286{K,PropertyID.PID_Demo_Max,ValueType.Scalar}@None
24.99999999999998{K,PropertyID.PID_Demo_Value,ValueType.Scalar}@None
